In [73]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from tqdm import tqdm

----
**Registro**

In [74]:
df = pd.read_csv("../DATA/df_clean_sin_duplicados.csv",index_col=0)

In [75]:
df.head()

,AGE,ATTRITION,BUSINESSTRAVEL,DEPARTMENT,DISTANCEFROMHOME,EDUCATION,EDUCATIONFIELD,EMPLOYEENUMBER,ENVIRONMENTSATISFACTION,GENDER,...,STOCKOPTIONLEVEL,TRAININGTIMESLASTYEAR,WORKLIFEBALANCE,YEARSATCOMPANY,YEARSSINCELASTPROMOTION,YEARSWITHCURRMANAGER,REMOTEWORK,DAILYRATE,PERFORMANCERATING,TOTALWORKINGYEARS
0,51.0,0,Unknown,Unknown,6,3,Unknown,162.0,1,male,...,0,5,3.0,20,15,15,1,684.0,3.0,11.223167
1,52.0,0,Unknown,Unknown,1,4,life sciences,259.0,3,male,...,1,5,3.0,33,11,9,1,699.0,3.0,34.000000
2,42.0,0,travel rarely,research & development,4,2,technical degree,319.0,3,male,...,0,3,3.0,22,11,15,1,532.0,3.0,22.000000
3,47.0,0,travel rarely,Unknown,2,4,medical,Unknown,1,female,...,2,2,3.0,20,5,6,0,359.0,3.0,10.971828
4,46.0,0,Unknown,Unknown,3,3,technical degree,Unknown,1,female,...,1,5,3.0,19,2,8,0,1319.0,3.0,11.714245


In [77]:
def ejecutar_query(query,valores):
    cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='RAW_DATA')

    mycursor = cnx.cursor()

    
    try:
        mycursor.execute(query,valores)
        cnx.commit()  # Confirma la transacción
        print("Query ejecutada correctamente.")

    except mysql.connector.Error as err:
        cnx.rollback()  # Revierte la transacción si hay un error
        print("Error al ejecutar la query:", err)

    # Cierra el cursor y la conexión a la base de datos
    mycursor.close()
    cnx.close()

In [78]:
def procesamiento_valor(row,nombre_columna):
    tipo_columna =  df[nombre_columna].dtype

    if nombre_columna == "EMPLOYEENUMBER" and row[nombre_columna] == "Unknown":
        return 0
    
    if tipo_columna == "int64":
        return int(row[nombre_columna])
    
    elif tipo_columna == "float64":
        return float(row[nombre_columna])
    
    return row[nombre_columna]

In [79]:
employe_number= []

inicio = 0
fin = len(df)-1

for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  # Obtén la fila en el índice actual

    if row['EMPLOYEENUMBER'] not in employe_number:
        employe_number.append(row['EMPLOYEENUMBER'])

       
        sql = "INSERT INTO empleado (EMPLOYEE_NUMBER, AGE, EDUCATION, EDUCATION_FIELD, GENDER, MARITAL_STATUS, WORK_LIFE_BALANCE, NUM_COMPANIES_WORKED, RELATIONSHIP_SATISFACTION, TOTAL_WORKING_YEARS) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = (procesamiento_valor(row,'EMPLOYEENUMBER'), procesamiento_valor(row,'AGE'), procesamiento_valor(row,'EDUCATION'), procesamiento_valor(row,'EDUCATIONFIELD'), procesamiento_valor(row,'GENDER'), procesamiento_valor(row,'MARITALSTATUS'), procesamiento_valor(row,'WORKLIFEBALANCE'), procesamiento_valor(row,'NUMCOMPANIESWORKED'), procesamiento_valor(row,'RELATIONSHIPSATISFACTION'), procesamiento_valor(row,'TOTALWORKINGYEARS'))

        ejecutar_query(sql,val)


        sql = "INSERT INTO empleado_empresa (EMPLOYEE_NUMBER, DEPARTMENT, BUSINESS_TRAVEL, JOB_INVOLVEMENT, JOB_ROLE, JOB_SATISFACTION, ATTRITION, YEARS_AT_COMPANY, MONTHLY_RATE, PERCENT_SALARY_HIKE, STOCK_OPTION_LEVEL, TRAINING_TIMES_LAST_YEAR, YEARS_SINCE_LAST_PROMOTION, YEARS_WITH_CURRENT_MANAGER, DAILY_RATE, PERFORMANCE_RATING) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = (procesamiento_valor(row,'EMPLOYEENUMBER'), procesamiento_valor(row,'DEPARTMENT'), procesamiento_valor(row,'BUSINESSTRAVEL'), procesamiento_valor(row,'JOBINVOLVEMENT'), procesamiento_valor(row,'JOBROLE'), procesamiento_valor(row,'JOBSATISFACTION'), procesamiento_valor(row,'ATTRITION'), procesamiento_valor(row,'YEARSATCOMPANY'), procesamiento_valor(row,'MONTHLYRATE'), procesamiento_valor(row,'PERCENTSALARYHIKE'), procesamiento_valor(row,'STOCKOPTIONLEVEL'), procesamiento_valor(row,'TRAININGTIMESLASTYEAR'), procesamiento_valor(row,'YEARSSINCELASTPROMOTION'), procesamiento_valor(row,'YEARSWITHCURRMANAGER'),  procesamiento_valor(row,'DAILYRATE'), procesamiento_valor(row,'PERFORMANCERATING'))

        ejecutar_query(sql,val)


    sql = "INSERT INTO registro (EMPLOYEE_NUMBER, DISTANCE_FROM_HOME, REMOTE_WORK, MONTHLY_INCOME, HOURLY_RATE, OVER_TIME, ENVIRONMENT_SATISFACTION) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val = (procesamiento_valor(row,'EMPLOYEENUMBER'), procesamiento_valor(row,'DISTANCEFROMHOME'), procesamiento_valor(row,'REMOTEWORK'), procesamiento_valor(row,'MONTHLYINCOME'), procesamiento_valor(row,'HOURLYRATE'), procesamiento_valor(row,'OVERTIME'), procesamiento_valor(row,'ENVIRONMENTSATISFACTION'))

    ejecutar_query(sql,val)


Insertando registros:   0%|          | 0/1560 [00:00<?, ?it/s]

Query ejecutada correctamente.
Query ejecutada correctamente.


Insertando registros:   0%|          | 1/1560 [00:02<1:02:37,  2.41s/it]

Query ejecutada correctamente.
Query ejecutada correctamente.
Query ejecutada correctamente.


Insertando registros:   0%|          | 2/1560 [00:04<1:01:48,  2.38s/it]

...

Query ejecutada correctamente.


Insertando registros: 100%|█████████▉| 1559/1560 [49:27<00:00,  1.26it/s]

Query ejecutada correctamente.


Insertando registros: 100%|██████████| 1560/1560 [49:28<00:00,  1.90s/it]

Query ejecutada correctamente.
